In [1]:
# !pip install torch
# !pip install transformers
# !pip install sentence-transformers

In [2]:
import pandas as pd

# f = pd.read_json(file_path, lines=True)
df_image_train = pd.read_json("data/train.jsonl", lines=True)
df_image_val = pd.read_json("data/dev.jsonl", lines=True)
df_image_test = pd.read_json("data/dev_seen.jsonl", lines=True)
# df_image_val.columns

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: [-1.37173804e-02 -4.28515561e-02 -1.56286024e-02  1.40537508e-02
  3.95537391e-02  1.21796280e-01  2.94333510e-02 -3.17524113e-02
  3.54960039e-02 -7.93140307e-02  1.75878033e-02 -4.04369645e-02
  4.97259721e-02  2.54912451e-02 -7.18699768e-02  8.14968497e-02
  1.47072645e-03  4.79627475e-02 -4.50335816e-02 -9.92174968e-02
 -2.81769224e-02  6.45045936e-02  4.44670171e-02 -4.76217456e-02
 -3.52952406e-02  4.38671671e-02 -5.28566018e-02  4.33030014e-04
  1.01921484e-01  1.64072271e-02  3.26996669e-02 -3.45986858e-02
  1.21339718e-02  7.94871300e-02  4.58339276e-03  1.57778822e-02
 -9.68210492e-03  2.87626311e-02 -5.05806357e-02 -1.55793801e-02
 -2.87907030e-02 -9.62281227e-03  3.15556228e-02  2.27349456e-02
  8.71449560e-02 -3.85027453e-02 -8.84718671e-02 -8.75497982e-03
 -2.12343298e-02  2.08924189e-02 -9.02078301e-02 -5.25732413e-02
 -1.05638728e-02  2.88311131e-02 -1.61454789e-02  6.17839675e-03
 -1.23234

In [4]:
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader
from transformers import AdamW
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, roc_auc_score

# Load the sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Assume we have some data
# texts = ['This is a good day!', 'I feel very sad', 'I am not happy about this', 'I am thrilled']
# labels = [1, 0, 0, 1]  # 1: Positive sentiment, 0: Negative sentiment
X_train = df_image_train['text']
y_train = df_image_train['label']

X_val = df_image_val['text']
y_val = df_image_val['label']

# X_test = df_image_test['text']
# y_test = df_image_test['label']

# # Split into training and testing sets
# train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

# Convert the texts to embeddings
train_embeddings = model.encode(X_train)
val_embeddings = model.encode(X_val)

# Convert everything into torch tensors
train_embeddings = torch.tensor(train_embeddings)
train_labels = torch.tensor(y_train)

val_embeddings = torch.tensor(val_embeddings)
val_labels = torch.tensor(y_val)

# A simple linear classifier on top of SBERT's embeddings
classifier = nn.Linear(train_embeddings.size(1), 2)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = AdamW(classifier.parameters(), lr=1e-2)

# Training loop
for epoch in range(300):  # for simplicity, we train for 10 epochs
    classifier.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = classifier(train_embeddings)
    loss = criterion(outputs, train_labels)

    # Backward pass
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

# Test the classifier
classifier.eval()

with torch.no_grad():
    outputs = classifier(val_embeddings)
    _, predicted = torch.max(outputs, 1)

# Compute precision, recall, and AUROC
precision = precision_score(val_labels, predicted, average='weighted')
recall = recall_score(val_labels, predicted, average='weighted')
roc_auc = roc_auc_score(val_labels, predicted)

print('Accuracy: ', (predicted == val_labels).sum().item() / val_labels.size(0))
print('Precision: ', precision)
print('Recall: ', recall)
print('AUROC: ', roc_auc)

2023-06-16 04:45:10.527986: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 04:45:11.667220: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


/home/musharma/.local/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 0.695159912109375
Epoch 2, Loss: 0.680543839931488
Epoch 3, Loss: 0.6547991037368774
Epoch 4, Loss: 0.6562486290931702
Epoch 5, Loss: 0.6419801712036133
Epoch 6, Loss: 0.6261335611343384
Epoch 7, Loss: 0.62160724401474
Epoch 8, Loss: 0.5995896458625793
Epoch 9, Loss: 0.5805856585502625
Epoch 10, Loss: 0.5945366621017456
Epoch 11, Loss: 0.5812563896179199
Epoch 12, Loss: 0.5955142378807068
Epoch 13, Loss: 0.5800560712814331
Epoch 14, Loss: 0.5738778710365295
Epoch 15, Loss: 0.5736756324768066
Epoch 16, Loss: 0.5637814402580261
Epoch 17, Loss: 0.5725136995315552
Epoch 18, Loss: 0.5644941329956055
Epoch 19, Loss: 0.5625475645065308
Epoch 20, Loss: 0.5624615550041199
Epoch 21, Loss: 0.5566096305847168
Epoch 22, Loss: 0.5610073804855347
Epoch 23, Loss: 0.5573152899742126
Epoch 24, Loss: 0.5548543334007263
Epoch 25, Loss: 0.5550036430358887
Epoch 26, Loss: 0.5505330562591553
Epoch 27, Loss: 0.553121030330658
Epoch 28, Loss: 0.5510523915290833
Epoch 29, Loss: 0.549892246723175


In [5]:
torch.save(model.state_dict(), f"sentence_transformer_final.pth")

In [6]:
# Test the classifier
classifier.eval()

df_image_test = pd.read_json("data/dev_seen.jsonl", lines=True)

X_test = df_image_test['text']
y_test = df_image_test['label']

test_embeddings = model.encode(X_test)

test_embeddings = torch.tensor(test_embeddings)
test_labels = torch.tensor(y_test)

with torch.no_grad():
    outputs = classifier(test_embeddings)
    _, predicted = torch.max(outputs, 1)

# Compute precision, recall, and AUROC
precision = precision_score(test_labels, predicted, average='weighted')
recall = recall_score(test_labels, predicted, average='weighted')
roc_auc = roc_auc_score(test_labels, predicted)

print('Accuracy: ', (predicted == test_labels).sum().item() / test_labels.size(0))
print('Precision: ', precision)
print('Recall: ', recall)
print('AUROC: ', roc_auc)

Accuracy:  0.55
Precision:  0.56677558910162
Recall:  0.55
AUROC:  0.5466947240402619
